In [ ]:
# Install additional packages to handle lemmatization.
!pip install symspellpy vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=187456 sha256=ceac04c29234173537bf81d4ce8021e2cb12c009355ae1b5e9d118d9552c5f16
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy


In [ ]:
# Change directory.
import os
os.chdir("/content/drive/MyDrive/ML Project 2")
os.getcwd()

'/content/drive/MyDrive/ML Project 2'

## Import dependencies

In [ ]:
from utility.paths import DataPath
from preprocessing import Preprocessing
from models.gru import GRU

import pandas as pd
from tqdm.auto import tqdm

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Create training and testing preprocessing object.
train_prep = Preprocessing([DataPath.TRAIN_NEG_FULL, DataPath.TRAIN_POS_FULL])
test_prep = Preprocessing([DataPath.TEST], is_test=True)

In [ ]:
# Declare params for GRU.
MAX_LEN = 256
BATCH_SIZE = 128
EPOCHS = 10
EMBEDDING_DIM = 100 # Since we're using GLoVe

In [ ]:
# Declare GRU model.
gru = GRU(weight_path=DataPath.GRU_WEIGHT,
          submission_path=DataPath.GRU_SUBMISSION,
          max_length=MAX_LEN)

## Preprocessing

In [ ]:
# Retrieve preprocessing steps declared in GRU class for both train and test data.
for step in tqdm(gru.preprocessing(), desc="Preprocessing train data"):
    getattr(train_prep, step)()

for step in tqdm(gru.preprocessing(is_train=False), desc="Preprocessing test data"):
    getattr(test_prep, step)()

Preprocessing train data:   0%|          | 0/14 [00:00<?, ?it/s]

Executing: `drop_duplicates`
Executing: `remove_ending`
Executing: `remove_extra_space`


100%|██████████| 2268591/2268591 [00:01<00:00, 1156548.06it/s]


Executing: `remove_space_around_emoji`
Executing: `remove_extra_space`


100%|██████████| 2268591/2268591 [00:01<00:00, 1178470.91it/s]


...

100%|██████████| 10000/10000 [00:00<00:00, 858626.38it/s]


Executing: `emoji_to_tag`


100%|██████████| 10000/10000 [00:00<00:00, 72363.25it/s]


Executing: `reconstruct_last_emoji`
Executing: `num_to_tag`
Executing: `hashtag_to_tag`
Executing: `repeat_symbols_to_tag`
Executing: `elongate_to_tag`
Executing: `remove_extra_space`


100%|██████████| 10000/10000 [00:00<00:00, 486990.61it/s]


In [ ]:
# Retrieve the preprocessed df.
train_data = train_prep.__get__()
test_data = test_prep.__get__()

In [ ]:
# Export the dataframes. For training frames, shuffles.
train_data = train_data.sample(frac=1)
train_data.to_csv(DataPath.GRU_TRAIN, index=False)

test_data.to_csv(DataPath.GRU_TEST, index=False)

## Training

In [ ]:
# Read the dataframe
train_df = pd.read_csv(DataPath.GRU_TRAIN)
train_df.head()

,text,label
0,i couldnt be happier <heart>,1.0
1,""" <user> why am i so <elong> tired "" maybe yu ...",0.0
2,yay ! ! ! rt <user> im gettting my award today...,1.0
3,<user> okie dokie ! try to stick around i wann...,1.0
4,not gonna speak to anyone at work the next cou...,0.0


In [ ]:
# Drop nan in the df
train_df.dropna(inplace=True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2268591 entries, 0 to 2268590
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   text    object 
 1   label   float64
dtypes: float64(1), object(1)
memory usage: 34.6+ MB


### Splitting data

In [ ]:
# Create X and y to feed into GRU
X, y = train_df['text'].values, train_df['label'].values

### Training Loop

In [ ]:
# Update vocabulary for GRU embedding
gru.update_vocabulary(X)

# Start the training process
gru.train(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS)

Executing: `update_vocabulary`
Vocabulary size: 439824
Executing: `padding`
Executing: `padding`
Executing: `generate_embedding_matrix`


Loading GloVe: 0it [00:00, ?it/s]

Found 1193514 word vectors


Generating embedding matrix:   0%|          | 0/439822 [00:00<?, ?it/s]

Converted 172879 words (266943 missing)
Executing: `build_model`
Model summary
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 256, 100)          43982400  
                                                                 
 bidirectional (Bidirection  (None, 200)               121200    
 al)                                                             
                                                                 
 dense (Dense)               (None, 100)               20100     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 44123801 (168.32 MB)
Trainable params: 141401 (552.35 KB)
Non-trainable params: 43982400 (167.78 MB)
____________________________________________________________

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saving weights


## Predicting

In [ ]:
# Read preprocessed test data
test_df = pd.read_csv(DataPath.GRU_TEST)
test_df.head()

,ids,text
0,1,sea doo pro sea scooter <sadface> sports with ...
1,2,<user> shucks well i work all week so now i ca...
2,3,i cant stay away from bug thats my baby
3,4,<user> no ma'am ! ! ! lol im perfectly fine an...
4,5,"whenever i fall asleep watching the tv , i alw..."


In [ ]:
# Retrieve `text` column for predicting
X_test = test_df["text"]
gru.predict(X_test)

Executing: `padding`
313/313 [==============================] - 4s 4ms/step


We can check in `submissions/gru` there is a submission file (`submission_2023-12-18_14:04:34.csv`). Once submit to AICrowd, it yields a score of `0.865` and secondary score of `0.866` (Submission #247060).